In [1]:
from sigvisa.models.spatial_regression.multi_shared_bcm import MultiSharedBCM, Blocker, sample_synthetic, vis
#from sigvisa.models.spatial_regression.local_regression import LocalGPs, BCM, MultiGPLVM

from sigvisa.treegp.gp import GPCov, GP, mcov, prior_sample, dgaussian
import scipy.stats
import scipy.optimize
import time

In [2]:
noise_var=0.01
X, Y, cov = sample_synthetic(n=400, noise_var=noise_var, yd=10)

In [11]:
pts = np.linspace(0, 1, 9)[1::2]
centers = [np.array((xx, yy)) for xx in pts for yy in pts]

b = Blocker(centers)
SX, SY, perm, block_boundaries = b.sort_by_block(X, Y)
YY = np.dot(SY, SY.T)

In [12]:
mbcm = MultiSharedBCM(SX, Y=SY, block_boundaries=block_boundaries, cov=cov, noise_var=noise_var, kernelized=False)
mbcmk = MultiSharedBCM(SX, Y=YY, block_boundaries=block_boundaries, cov=cov, noise_var=noise_var, kernelized=True, dy=10)

In [13]:
print mbcm.llgrad(grad_X=False)
print mbcmk.llgrad()

(-557.68437488450945, array([], dtype=float64))
(-1717.9629294583647, array([], dtype=float64))


In [15]:
print mbcmk.llgrad_unary(0)
print mbcm.llgrad_unary(0)

(-131.89547469593759, array([], dtype=float64))
(-91.293088227403217, array([], dtype=float64))


In [17]:
i_start, i_end = mbcm.block_boundaries[0]
bYY = mbcmk.YY[i_start:i_end, i_start:i_end]
bY = mbcm.Y[i_start:i_end]
bYYb = np.dot(bY, bY.T)
print bYY
print b

[[  9.14443265e+00   8.44102008e-01   3.91799597e+00   6.28068437e+00
    3.64102719e+00  -7.30117297e+00  -2.97623292e-01   2.96617300e-01
    1.88225239e+00  -7.80582038e+00   8.13418558e+00   7.51104787e+00
    8.92972963e+00   1.73750691e+00   3.68731335e+00   1.79294619e+00
   -4.81260878e+00  -7.61313645e+00   6.03389688e+00   7.26791758e-01
    1.98230811e+00  -1.31276256e+00  -5.70767849e+00   2.70162004e+00
    8.45401778e+00  -3.54858627e-01   1.45962069e+00]
 [  8.44102008e-01   3.93386553e+00  -2.06310997e+00   3.05337489e+00
   -3.11477036e+00   3.35026814e+00  -3.18305582e-01  -1.19371779e+00
    4.08435562e-01   2.09192871e+00   2.97936822e+00   2.25140637e+00
   -1.26233832e-01  -3.26096975e+00   2.64875466e+00   7.15960095e-01
    4.40023485e+00   4.01820961e+00  -1.13453791e+00  -1.27529229e-01
    1.34244960e+00  -1.88160468e+00   4.41623292e+00   3.70283249e+00
    1.45784285e+00   3.92033574e+00  -2.56013117e-01]
 [  3.91799597e+00  -2.06310997e+00   9.87796774e+00

In [26]:
from sigvisa.models.spatial_regression.local_regression import LocalGPs, BCM, MultiGPLVM

lscale=0.1
mlgps = MultiGPLVM(X=SX, Y=SY, cov_block_params = np.array(([noise_var, 1.0, lscale, lscale],)), 
                   bcm=True, block_centers=centers, test_cov=cov)

In [27]:
print mlgps.likelihood()
print mlgps.llgrad_X()

-557.684466649
[[  257.46289495   278.68160316]
 [  -84.84509835  -134.95428601]
 [ -300.94357966    57.55191219]
 [  204.69556726   488.74362757]
 [   96.29073947    20.11528082]
 [  179.92817661  -400.40845382]
 [ -104.89030029   313.39304723]
 [  105.21016121   -82.39805242]
 [ -216.17640112  -345.79354387]
 [ -172.21956352   178.11771485]
 [  144.29560261   114.52957775]
 [  119.87575735  -498.95170404]
 [ -517.63320446  -255.90394318]
 [  -94.66376744   148.82407497]
 [ -275.26041946  -178.5808985 ]
 [  251.18387208   136.14591684]
 [  -92.71178518   165.28588453]
 [  157.61036834   -17.60985175]
 [  386.62209024   -66.02270996]
 [ -295.97257474  -421.86280874]
 [   50.76262417   -88.28454075]
 [  -40.07861437  -272.19497474]
 [ -321.83265368  -412.85740578]
 [ -633.92927566   202.41177682]
 [  -20.14298835  -273.9985981 ]
 [  738.99860455   410.20915889]
 [  699.56816603   718.4469381 ]
 [   13.86090269    47.70467723]
 [   65.21638549   223.9119871 ]
 [  143.76466269   385.99125

In [14]:
print mlgps.lgps[0].gps[0].log_likelihood()
print mlgps.lgps[0].gps[0].X

-7.44073098458
[[ 0.14675589  0.09233859]
 [ 0.14038694  0.19810149]
 [ 0.08504421  0.03905478]
 [ 0.17234051  0.13713575]
 [ 0.01988013  0.02621099]
 [ 0.02830649  0.24621107]
 [ 0.05991769  0.12134346]
 [ 0.04455188  0.10749413]
 [ 0.15679139  0.0185762 ]
 [ 0.01395157  0.23436209]
 [ 0.17395567  0.12632952]
 [ 0.23366614  0.10196726]
 [ 0.14342305  0.0783869 ]
 [ 0.01833264  0.066725  ]
 [ 0.14555823  0.15773007]
 [ 0.24985068  0.17343017]
 [ 0.08748221  0.22730974]
 [ 0.0569644   0.22881367]
 [ 0.0935771   0.06537172]
 [ 0.13766274  0.01088873]
 [ 0.21411214  0.19894792]
 [ 0.02662788  0.11092037]
 [ 0.08052953  0.23170231]
 [ 0.15983365  0.20260213]
 [ 0.20645406  0.07618881]
 [ 0.12076718  0.22028399]
 [ 0.1443154   0.01409229]]


In [15]:
print mbcm.llgrad_unary(0)
i_start, i_end = mbcm.block_boundaries[0]
print mbcm.X[i_start:i_end, :]

(-33.180889277311842, array([], dtype=float64))
[[  4.17022005e-01   7.20324493e-01]
 [  1.14374817e-04   3.02332573e-01]
 [  1.46755891e-01   9.23385948e-02]
 [  1.86260211e-01   3.45560727e-01]
 [  3.96767474e-01   5.38816734e-01]
 [  4.19194514e-01   6.85219500e-01]
 [  2.04452250e-01   8.78117436e-01]
 [  2.73875932e-02   6.70467510e-01]
 [  4.17304802e-01   5.58689828e-01]
 [  1.40386939e-01   1.98101489e-01]
 [  8.00744569e-01   9.68261576e-01]
 [  3.13424178e-01   6.92322616e-01]
 [  8.76389152e-01   8.94606664e-01]
 [  8.50442114e-02   3.90547832e-02]
 [  1.69830420e-01   8.78142503e-01]
 [  9.83468338e-02   4.21107625e-01]
 [  9.57889530e-01   5.33165285e-01]
 [  6.91877114e-01   3.15515631e-01]
 [  6.86500928e-01   8.34625672e-01]
 [  1.82882773e-02   7.50144315e-01]
 [  9.88861089e-01   7.48165654e-01]
 [  2.80443992e-01   7.89279328e-01]
 [  1.03226007e-01   4.47893526e-01]
 [  9.08595503e-01   2.93614148e-01]
 [  2.87775339e-01   1.30028572e-01]
 [  1.93669579e-02   6.7883

In [11]:
ll = 0
def ll(X, Y):
    KK = mcov(X, cov, noise_var) 
    rv = scipy.stats.multivariate_normal(cov=KK)
    ll = 0
    for y in Y.T:
        ll1 = rv.logpdf(y)
        ll += ll1
    return ll

X0 = X_blocks[0]
x0  = X0.flatten()
def llg(xx):
    X = xx.reshape(X0.shape)
    return ll(X, Y_blocks[0])

print scipy.optimize.approx_fprime(x0, llg, 1e-8)

[  3.96777715e+02   1.90332372e+02  -9.39526686e+01   6.81998813e-01
  -2.88786495e+02   4.97046457e+01   1.46224305e+02   4.08451234e+02
   1.02764888e+02   1.17450512e+01  -2.67354878e+01  -5.70615626e+01
  -1.46210343e+02   2.80372002e+02   1.26232220e+02  -8.74668288e+01
  -2.38314573e+02  -4.84558332e+02  -1.13487114e+02  -4.79737565e+00
   1.78696973e+02  -4.00912938e+00  -1.66220252e+02   1.35212339e+02
  -5.95530588e+02  -3.28017396e+02  -9.44813848e+01   1.37684682e+02
  -1.61188302e+02  -2.11355164e+02   1.60333329e+01  -1.36423591e+02
  -6.59401422e+01   9.95629264e+01   1.12011499e+02  -1.06953408e+01
   3.40118142e+02   4.25605364e+01  -3.61328686e+02  -3.16985508e+02
   1.95608011e+02   1.21041381e+02  -4.40058940e+01  -2.24046485e+02
  -2.13465742e+02  -2.02001218e+02  -6.87965036e+02   3.59269734e+01
   7.96373641e+01  -2.64447947e+02   8.61101843e+02   1.27557345e+02
   7.42394725e+02   6.91021022e+02]


In [32]:
XX = X_blocks[0]
YY = Y_blocks[0][:, 0:1]
n, dx = XX.shape
dy = YY.shape[1]

K = mcov(XX, cov, noise_var)
prec = np.linalg.inv(K)
Alpha = np.dot(prec, YY)

ll = -.5 * np.sum(YY*Alpha)
ll += -.5 * dy * np.linalg.slogdet(K)[1]
ll += -.5 * dy * n * np.log(2*np.pi)

llgrad = np.zeros((n, dx))
for p in range(n):
    for i in range(dx):
        dll = 0
        dcv = mbcm.dKdx(XX, p, i, return_vec=True)
        #t1 = -np.outer(prec[p,:], dcv)
        #t1[:, p] = -np.dot(prec, dcv)
        #dll_dcov = .5*ny*np.trace(t1)

        dll_dcov = -dy * np.dot(prec[p,:], dcv)
        
        
        for j in range(dy):
            alpha = Alpha[:,j]
            dK_alpha = dcv * alpha[p]
            dK_alpha[p] = np.dot(dcv, alpha)
            dll_dcov = .5*np.dot(alpha, dK_alpha)
            dll += dll_dcov
            
        llgrad[p, i] = dll_dcov
        
print llgrad

[[  3.3398819   -3.49178491]
 [  5.55437133   6.24502219]
 [  6.00328332  14.51406571]
 [ -9.14623588  -3.4393161 ]
 [  8.04056792  23.25758416]
 [ -2.32176946 -44.14360124]
 [-11.5132134  -12.00542594]
 [ -4.40823803   6.30854935]
 [-23.74392705   1.75693795]
 [ 26.24540291  23.03947336]
 [ -8.42059923   1.12571558]
 [-20.98135396  -2.30624784]
 [ -1.18536019   6.9502817 ]
 [ 11.4355909   -0.35791765]
 [ 13.92375319   1.97622617]
 [-20.26493014   1.22194038]
 [ -1.26437493   0.7055501 ]
 [  7.63525814  22.74111107]
 [  2.28300849  -7.39490067]
 [ 14.39360812  18.23213263]
 [ -4.29422879 -16.49873024]
 [ 27.53431395 -16.92219381]
 [ -2.8534925  -15.26940264]
 [ -8.82685294  -9.18359181]
 [ -2.09211009  15.29371542]
 [ -6.85927946 -16.98530375]
 [  1.78692589   4.63011082]]


In [33]:
X0 = X_blocks[0]
x0 = X0.flatten()
def logdet(xx):
    XX = xx.reshape(X0.shape)
    K = mcov(XX, cov, noise_var)
    return -.5 * dy * np.linalg.slogdet(K)[1]

print scipy.optimize.approx_fprime(x0, logdet, 1e-6)


[  3.33977664  -3.49185975   5.55414833   6.24500116   6.00334491
  14.51408001  -9.14646315  -3.43937771   8.04052881  23.25776216
  -2.32177574 -44.14338694 -11.51310611 -12.00558865  -4.40832011
   6.30825488 -23.7441649    1.75644038  26.24542174  23.03943562
  -8.42077915   1.125691   -20.98132124  -2.30616974  -1.18537568
   6.9501728   11.43531325  -0.35753906  13.92380109   1.97640155
 -20.26488703   1.22204624  -1.26446803   0.70391278   7.63547642
  22.74062889   2.28305754  -7.39480518  14.39339142  18.23110353
  -4.29403268 -16.49891811  27.53427741 -16.92222444  -2.85359613
 -15.27109209  -8.82669146  -9.18397459  -2.09197502  15.29378217
  -6.85928764 -16.98614582   1.78629197   4.62884647]


[-0.14868991  2.45489368  0.14641511 -2.44564378]


In [21]:
print  -.5 * np.sum(YY*Alpha)
print -.5 * np.dot(YY.T, np.dot(prec, YY))

-12.5160292931
[[-12.51602929]]
2
